In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file2=pd.read_csv("Sentiment_Analysis Dataset.csv",error_bad_lines=False)

file2

In [ ]:
from sklearn.model_selection import train_test_split
Text=list(file2.SentimentText)
file2.groupby('Sentiment').count()
train_data,test_data,train_label,test_label=train_test_split(Text, file2.Sentiment, test_size=0.4, random_state=42)

In [ ]:
vocabDic=dict()
for document in train_data:
    document=document.split(" ")
    for word in document:
        if word not in vocabDic:
            vocabDic[word]=len(vocabDic)+1
vocabDic['unk']=len(vocabDic)+1
    

In [ ]:
train_data[0]


In [ ]:
def codeDocuments(documents,dictionnary):
    documentsArray=list()
    for i,document in enumerate(documents):
        tempList=list()
        document=document.split(" ")
        for word in document:
            if word in vocabDic:
                word=vocabDic[word]
            else:
                word=vocabDic['unk']
            tempList.append(word)
        documentsArray.append(tempList)
    return np.array(documentsArray)
train_docs=codeDocuments(train_data,vocabDic)
test_docs=codeDocuments(test_data,vocabDic)

In [ ]:
train_docs

In [ ]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

avgList=mean([len(x) for x in train_docs])
print(avgList)
max([len(x) for x in train_docs])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing import sequence

maxlen=75
train_set = sequence.pad_sequences(train_docs, maxlen=maxlen)
test_set = sequence.pad_sequences(test_docs, maxlen=maxlen)

mlb = MultiLabelBinarizer()
#test_labels_encoded=mlb.fit_transform(train_label)
#train_labels_encoded=mlb.fit_transform(test_label)
type(train_label)

In [ ]:
train_set

In [ ]:
train_label=np.array(train_label)
test_label=np.array(test_label)
len(train_label)
len(test_label)
train_label

Using Kim's CNN : https://arxiv.org/pdf/1408.5882.pdf

In [ ]:
from keras.layers import Embedding
from keras.layers import Conv2D
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras import backend as K

from keras.models import Model
import tensorflow as tf
from sklearn import preprocessing
import keras
import numpy as np
import pdb

#using multiple filters
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.7
 
# Create a session with the above options specified.
config.gpu_options.per_process_gpu_memory_fraction = 0.65
K.tensorflow_backend.set_session(tf.Session(config=config))
class Classifier():
    def __init__(self,Vocab,maxlen=75):
        self.model=Sequential()
        self.EMBED_DIM=30
        self.batch_size=30
        self.MAX_SEQUENCE_LENGTH=maxlen
        self.nb_labels=2
        self.model = Sequential()
        self.Vocab=Vocab
    def fit(self, X,y):
        #pdb.set_trace()
        
        mainIn=Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32', name='main_input')
        
        x=Embedding(len(self.Vocab)+2,self.EMBED_DIM,input_length=self.MAX_SEQUENCE_LENGTH)(mainIn)
        
        x=Reshape((1,self.MAX_SEQUENCE_LENGTH, self.EMBED_DIM))(x)
        
        x1=Conv2D(128, strides=2,kernel_size=5 ,activation="relu", padding='same')(x)

        x1=MaxPooling2D((self.MAX_SEQUENCE_LENGTH-5+1,1),padding='same')(x1)
        x1=Flatten()(x1)
        x2=Conv2D(128, strides=2, kernel_size=4, activation="sigmoid", padding='same')(x)
        x2=MaxPooling2D((self.MAX_SEQUENCE_LENGTH-4+1,1),padding='same')(x2)
        
        x2=Flatten()(x2)
        x3=Conv2D(128, strides=2, kernel_size=3, activation="tanh", padding='same')(x)
        x3=MaxPooling2D((self.MAX_SEQUENCE_LENGTH-3+1,1),padding='same')(x3)
        
        x3=Flatten()(x3)
        
        combinedX=keras.layers.concatenate([x1,x2,x3],axis=1)
        
        
        combinedX=Dense(128, activation="relu")(combinedX)
        combinedX=Dropout(0.2)(combinedX)
        #output=Dense(self.nb_labels, activation="sigmoid")(combinedX)
        output=Dense(2, activation="softmax")(combinedX)
        #output=Dense(1, activation="sigmoid")(combinedX)

        
        encoder =preprocessing.LabelEncoder()
        encoder.fit(y)
        encoded_Y = encoder.transform(y)
        labels=keras.utils.to_categorical(encoded_Y, num_classes=2)
        #labels=y
        inputs2=X
        self.model = Model(inputs=mainIn, outputs=output)
        
       # self.model.compile(loss='binary_crossentropy',
       #                 optimizer='adam',
       #                 metrics=['acc'])
        
        self.model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['acc'])
        
        self.model.fit(inputs2,labels,epochs=7, batch_size=self.batch_size)
    def predict(self, X):
        return self.model.predict(np.array(X))

    def predict_proba(self, X):
        
        return self.model.predict(np.array(X), self.batch_size)



In [ ]:
model=Classifier(vocabDic,maxlen)

In [ ]:
model=Classifier(vocabDic,maxlen)
model.fit(train_set[:50000],train_label[:50000])


#model.fit(train_set[:50],train_label[:50])


In [ ]:
del model

In [ ]:
predictions=model.predict(test_set)
predicted_labels=np.argmax(predictions,axis=1)
correct_predictions=sum(predicted_labels==test_label)
print(correct_predictions/len(test_label))

Using a simple LSTM

In [ ]:
from keras.layers import Embedding
from keras.layers import Conv2D
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras import backend as K
from keras.layers import LSTM
from keras.models import Model
import tensorflow as tf
from sklearn import preprocessing
import keras
import numpy as np
import pdb

#using multiple filters
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
config.gpu_options.per_process_gpu_memory_fraction = 0.5
K.tensorflow_backend.set_session(tf.Session(config=config))
class ClassifierLSTM():
    def __init__(self,Vocab,maxlen):
        self.model=Sequential()
        self.EMBED_DIM=30
        self.batch_size=30
        self.MAX_SEQUENCE_LENGTH=maxlen
        self.nb_labels=2
        self.model = Sequential()
        self.Vocab=Vocab
    def useWeights(self,embed_dim,X):
        embeddings_index = {}
        VOCAB=X['statements'][1].keys()
        vocabDic=VOCAB=X['statements'][1]
        EMBEDDING_DIM=embed_dim
        embedding_matrix = np.zeros((len(VOCAB) + 1, EMBEDDING_DIM))
        i=0
        with open('wiki-news-300d-1M.vec') as f:
            next(f)
            for line in f:
                values = line.split()
                word = values[0]

                if word in VOCAB:
                    embedding_matrix[vocabDic[word],:]=np.asarray(values[1:EMBEDDING_DIM+1], dtype='float32')
                    i=i+1
        return (embedding_matrix)
    
    def fit(self, X,y):
        #pdb.set_trace()
        
        mainIn=Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32', name='main_input')
        
        x=Embedding(len(self.Vocab)+2,self.EMBED_DIM,input_length=self.MAX_SEQUENCE_LENGTH)(mainIn)
        
        #x=Reshape((1,self.MAX_SEQUENCE_LENGTH, self.EMBED_DIM))(x)
        
        x=LSTM(self.EMBED_DIM, dropout=0.2, recurrent_dropout=0.2)(x)
        output=Dense(1, activation='sigmoid')(x)
        
        #encoder =preprocessing.LabelEncoder()
        #encoder.fit(y)
        #encoded_Y = encoder.transform(y)
        #labels=keras.utils.to_categorical(encoded_Y, num_classes=2)
        labels=y
        inputs=X
        self.model = Model(inputs=mainIn, outputs=output)
        
        self.model.compile(loss='binary_crossentropy',
                        optimizer='adam',
                        metrics=['acc'])
        self.model.summary
        self.model.fit(inputs,labels,epochs=3, batch_size=self.batch_size)
    def predict(self, X):
        return self.model.predict(np.array(X))

    def predict_proba(self, X):
        
        return self.model.predict(np.array(X), self.batch_size)



In [ ]:
model=ClassifierLSTM(vocabDic,maxlen)
model.fit(train_set[:500000],train_label[:500000])
predictions=model.predict(test_set)
predicted_labels=np.argmax(predictions,axis=1)
correct_predictions=sum(predicted_labels==test_label)
print(correct_predictions/len(test_label))

In [ ]:
predictions_bool=predictions>0.5
predictions_bool=[1 if x ==True else 0 for x in predictions_bool]

In [ ]:


predicted_labels=predictions>0.5
correct_predictions=sum(predictions_bool==test_label)
print(correct_predictions/len(test_label))

In [ ]:
from keras.layers import Embedding
from keras.layers import Conv2D
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Bidirectional
from keras import backend as K
from keras.layers import LSTM
from keras.models import Model
import tensorflow as tf
from sklearn import preprocessing
import keras
import numpy as np
import pdb

#using multiple filters
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
config.gpu_options.per_process_gpu_memory_fraction = 0.5
K.tensorflow_backend.set_session(tf.Session(config=config))
class ClassifierBidirectionalLSTM():
    def __init__(self,Vocab,maxlen):
        self.model=Sequential()
        self.EMBED_DIM=30
        self.batch_size=30
        self.MAX_SEQUENCE_LENGTH=maxlen
        self.nb_labels=2
        self.model = Sequential()
        self.Vocab=Vocab
    def useWeights(self,embed_dim,X):
        embeddings_index = {}
        VOCAB=X['statements'][1].keys()
        vocabDic=VOCAB=X['statements'][1]
        EMBEDDING_DIM=embed_dim
        embedding_matrix = np.zeros((len(VOCAB) + 1, EMBEDDING_DIM))
        i=0
        with open('wiki-news-300d-1M.vec') as f:
            next(f)
            for line in f:
                values = line.split()
                word = values[0]

                if word in VOCAB:
                    embedding_matrix[vocabDic[word],:]=np.asarray(values[1:EMBEDDING_DIM+1], dtype='float32')
                    i=i+1
        return (embedding_matrix)
    
    def fit(self, X,y):
        #pdb.set_trace()
        
        mainIn=Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32', name='main_input')
        
        x=Embedding(len(self.Vocab)+2,self.EMBED_DIM,input_length=self.MAX_SEQUENCE_LENGTH)(mainIn)
        
        #x=Reshape((1,self.MAX_SEQUENCE_LENGTH, self.EMBED_DIM))(x)
        
        x=Bidirectional(LSTM(self.EMBED_DIM, dropout=0.2, recurrent_dropout=0.2))(x)
        output=Dense(1, activation='sigmoid')(x)
        
        #encoder =preprocessing.LabelEncoder()
        #encoder.fit(y)
        #encoded_Y = encoder.transform(y)
        #labels=keras.utils.to_categorical(encoded_Y, num_classes=2)
        labels=y
        inputs=X
        self.model = Model(inputs=mainIn, outputs=output)
        
        self.model.compile(loss='binary_crossentropy',
                        optimizer='adam',
                        metrics=['acc'])
        self.model.summary
        self.model.fit(inputs,labels,epochs=7, batch_size=self.batch_size)
    def predict(self, X):
        return self.model.predict(np.array(X))

    def predict_proba(self, X):
        
        return self.model.predict(np.array(X), self.batch_size)



In [ ]:
model=ClassifierBidirectionalLSTM(vocabDic,maxlen)
model.fit(train_set[:500000],train_label[:500000])
predictions=model.predict(test_set)
predictions_bool=predictions>0.5
predictions_bool=[1 if x ==True else 0 for x in predictions_bool]
correct_predictions=sum(predictions_bool==test_label)
print(correct_predictions/len(test_label))
